In [14]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_8\Log\training.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_8\Data\Preprocessing\Preprosessed.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [15]:
import numpy as np
import pandas as pd

# Original unique kilometr qiymatlari (13 ta)
unique_km = sorted(df['kilometer'].unique())

# Continuous target yaratish
continuous_km = []

# Har bir original qiymat oralig'ini aniqlaymiz
for i, km in enumerate(unique_km):
    if i == len(unique_km) - 1:
        # Oxirgi qiymat uchun yuqori limit qo'yamiz (masalan +10000)
        upper = km + 10000
    else:
        upper = unique_km[i + 1]
    
    # Har bir row uchun random continuous qiymat yaratish
    mask = df['kilometer'] == km
    n_rows = mask.sum()
    
    # Linear space oralig'ida random values yaratish
    values = np.random.uniform(low=km, high=upper, size=n_rows)
    continuous_km.extend(values)

# Continuous target ustuni yaratish
df['kilometer_cont'] = continuous_km

# Tekshirish
print(df[['kilometer', 'kilometer_cont']].head(20))
print("Unique values in continuous target:", df['kilometer_cont'].nunique())


    kilometer  kilometer_cont
0    1.000000        0.005000
1    0.827586        0.010432
2    0.827586        0.009655
3    1.000000        0.004619
4    0.586207        0.014621
5    1.000000        0.032116
6    1.000000        0.014768
7    0.241379        0.012013
8    0.172414        0.017572
9    1.000000        0.019944
10   1.000000        0.013605
11   1.000000        0.020727
12   1.000000        0.012077
13   1.000000        0.004152
14   0.448276        0.025320
15   0.000000        0.013374
16   1.000000        0.004970
17   1.000000        0.013903
18   1.000000        0.000287
19   1.000000        0.012065
Unique values in continuous target: 371528


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 28 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              371528 non-null  int64  
 1   index                   371528 non-null  float64
 2   dateCrawled             371528 non-null  float64
 3   name                    371528 non-null  float64
 4   price                   371528 non-null  float64
 5   vehicleType             371528 non-null  float64
 6   yearOfRegistration      371528 non-null  float64
 7   powerPS                 371528 non-null  float64
 8   model                   371528 non-null  float64
 9   kilometer               371528 non-null  float64
 10  monthOfRegistration     371528 non-null  float64
 11  fuelType                371528 non-null  float64
 12  brand                   371528 non-null  float64
 13  dateCreated             371528 non-null  float64
 14  nrOfPictures        

In [17]:
df.drop(columns=['kilometer'], inplace=True)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371528 entries, 0 to 371527
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              371528 non-null  int64  
 1   index                   371528 non-null  float64
 2   dateCrawled             371528 non-null  float64
 3   name                    371528 non-null  float64
 4   price                   371528 non-null  float64
 5   vehicleType             371528 non-null  float64
 6   yearOfRegistration      371528 non-null  float64
 7   powerPS                 371528 non-null  float64
 8   model                   371528 non-null  float64
 9   monthOfRegistration     371528 non-null  float64
 10  fuelType                371528 non-null  float64
 11  brand                   371528 non-null  float64
 12  dateCreated             371528 non-null  float64
 13  nrOfPictures            371528 non-null  float64
 14  postalCode          

In [19]:
df.to_csv(r"C:\Users\Rasulbek907\Desktop\Project_8\Data\Preprocessing\feature_km.csv")

In [4]:
from IPython.display import display, HTML

display(HTML("""
<a href="file:///C:/Users/Rasulbek907/Desktop/Project_8/Data/Preprocessing/feature_km.csv" target="_blank" style="
    display: inline-block;
    padding: 10px 20px;
    font-size: 16px;
    color: white;
    background-color: #FF5722;
    text-align: center;
    text-decoration: none;
    border-radius: 5px;
">
Open Filtered Features CSV
</a>
"""))


# Kilometer Target Yondashuvlari

## 1️⃣ 13 ta diskret qiymat bilan regressiya (`kilometer_reg`)

**Afzalliklari:**

- Oddiy va aniq: asl datasetdagi 13 ta qiymat bilan ishlaydi.  
- Model “classification-like” regressiya qiladi.  
- Kamroq o‘rganish kerak, oson va tezroq.  

**Kamchiliklari:**

- Target **diskret**, shuning uchun haqiqiy continuous regressiya modellarining afzalliklari (masalan, linear trend, interpolation) ishlamaydi.  
- Modelning natijalari ham 13 ta qiymatdan tashqariga chiqmaydi.  
- Agar maqsad haqiqiy km bashorat qilish bo‘lsa, bu cheklovchi.  

---

## 2️⃣ Continuous target (`kilometer_cont`)

**Afzalliklari:**

- Haqiqiy regressiya uchun mos: model haqiqiy km qiymatlarni taxmin qilishi mumkin.  
- Random yoki interval asosida continuous qilinganligi sababli, model interpolatsiya qilishi mumkin.  
- Regressiya metrics (MAE, RMSE) yanada “realistik” bo‘ladi.  

**Kamchiliklari:**

- Continuous target random qilingan, ya’ni bir qismda shovqin (noise) bor.  
- Model o‘rgangan narsa “ma’lumotga qo‘shilgan randomness” bo‘lishi mumkin.  
- Agar datasetda haqiqiy km faqat 13 ta kategoriyada bo‘lsa, continuous qilish ba’zan “asliy ma’lumotni o‘zgartirish” hisoblanadi.  

---

## 🔹 Amaliy tavsiya

| Maqsad | Qaysi yondashuv |
|--------|----------------|
| Siz haqiqiy km ni bashorat qilmoqchisiz (masalan mashina qaysi km da ekanini taxmin qilish) | Continuous target (`kilometer_cont`) |
| Siz faqat km diapazonini (0–5k, 5–10k, 10–20k…) aniqlamoqchisiz | Diskret target (`kilometer_reg`) |

In [20]:
df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_8\Data\Preprocessing\feature_km.csv")

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# Regression models
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
# from sklearn.svm import SVR
import xgboost as xgb 

# --------------------------
# 1️⃣ X va y ajratish
# --------------------------
target_col = 'kilometer_cont'
X = df.drop(target_col, axis=1)
y = df[target_col]

# Train / Validation / Test (60/20/20)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# --------------------------
# 2️⃣ Modellarni yaratish
# --------------------------
models = {
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Linear Regression": LinearRegression(),
    "KNN Regressor": KNeighborsRegressor(n_neighbors=5),
    # "SVR": SVR(),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

# --------------------------
# 3️⃣ Model trening va baholash
# --------------------------
results = []

for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    # Predict
    y_pred_val = model.predict(X_val)
    y_pred_test = model.predict(X_test)
    
    # Baholash
    r2_val = r2_score(y_val, y_pred_val)
    mse_val = mean_squared_error(y_val, y_pred_val)
    r2_test = r2_score(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
     
    results.append({
        "Model": name,
        "R2_Validation": r2_val,
        "MSE_Validation": mse_val,
        "R2_Test": r2_test,
        "MSE_Test": mse_test
    })

# --------------------------
# 4️⃣ Natijalarni jadval shaklida chiqarish
# --------------------------
results_df = pd.DataFrame(results)
print(results_df.sort_values(by="R2_Validation", ascending=False))

               Model  R2_Validation  MSE_Validation   R2_Test      MSE_Test
4            XGBoost       0.513549    5.404332e+06  0.514259  5.386329e+06
0      Random Forest       0.503607    5.514777e+06  0.505278  5.485914e+06
3      KNN Regressor       0.414279    6.507188e+06  0.415860  6.477468e+06
2  Linear Regression       0.350926    7.211020e+06  0.353131  7.173060e+06
1      Decision Tree       0.011635    1.098045e+07  0.015696  1.091484e+07


In [1]:
from IPython.display import display, HTML

# Clickable link (button ko‘rinishida)
display(HTML("""
<a href="feature_selection.ipynb" target="_self" style="
    display: inline-block;
    padding: 10px 20px;
    font-size: 16px;
    color: white;
    background-color: #4CAF50;
    text-align: center;
    text-decoration: none;
    border-radius: 5px;
">
Next Step
</a>
"""))
